In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from IPython.display import display

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
from time import sleep
import random
from seleniumwire.utils import decode
import json

from bs4 import BeautifulSoup
from urllib.parse import urlparse

from tqdm.auto import tqdm, trange
import pandas as pd
import time
import os
import seleniumwire.undetected_chromedriver as uc
import traceback
from collections import defaultdict

import re

import requests

import numpy as np

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor

In [2]:
def create_driver():
    chrome_options = uc.ChromeOptions()
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--blink-settings=imagesEnabled=false')
    chrome_options.page_load_strategy = 'none'

    driver = uc.Chrome(
            options=chrome_options,
            seleniumwire_options={}
    )
    return driver

## Базовый пример

In [17]:
url = 'https://www.ozon.ru/product/shvabra-daris-dlina-120-sm-220337295'

# Open webpage
driver = create_driver()
driver.get(url)

# It takes some time to load the page, we need to wait for requests that we want to parse
sleep(5)

# This contains all the requests page was able to make once the sleep has passed
driver.requests

[Request(method='POST', url='https://accounts.google.com/ListAccounts?gpsia=1&source=ChromiumBrowser&json=standard', headers=[('content-length', '1'), ('origin', 'https://www.google.com'), ('content-type', 'application/x-www-form-urlencoded'), ('sec-fetch-site', 'none'), ('sec-fetch-mode', 'no-cors'), ('sec-fetch-dest', 'empty'), ('user-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'), ('accept-encoding', 'gzip, deflate, br'), ('accept-language', 'en-US,en;q=0.9')], body=b' '),
 Request(method='GET', url='https://clientservices.googleapis.com/chrome-variations/seed?osname=mac&channel=stable&milestone=117', headers=[('a-im', 'gzip'), ('sec-fetch-site', 'none'), ('sec-fetch-mode', 'no-cors'), ('sec-fetch-dest', 'empty'), ('user-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'), ('accept-encoding', 'gzip, deflate, br')], body=b''),


## Пример использования на сайте DNS

In [3]:
regions = ['Москва', 'Санкт-Петербург', 'Казань']
shops_url='https://shops.dns-shop.ru/v1/shops-avail-by-partition'

siteId = "8c2e120b-8732-48a7-8178-0e04d47962d8"
cityIds = {
    "Москва" : "30b7c1f3-03fb-11dc-95ee-00151716f9f5",
    "Санкт-Петербург" : "566ca284-5bea-11e2-aee1-00155d030b1f",
    "Казань" : "55506b55-0565-11df-9cf0-00151716f9f5",
}
city_paths = {
    "Москва" : "moscow",
    "Санкт-Петербург" : "spb",
    "Казань" : "kazan",
}

In [4]:
def get_price_page(microdata_request, city_path):
    cookie = microdata_request.headers['Cookie'].split(' ')
    for i in range(len(cookie)):
        if 'city_path' in cookie[i]:
            cookie[i] = f'city_path={city_path};'
    headers = dict(microdata_request.headers)
    headers['Cookie'] = ' '.join(cookie)
    
    response = requests.get(microdata_request.url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f'smth wrong with price request: Status Code is {response.status_code}') 

    return response.json()

In [5]:
def get_price(price_page):
    price = priceCurrency = None
    if 'offers' in price_page['data'].keys() and 'price' in price_page['data']['offers'].keys():
        price = price_page['data']['offers']['price']
        priceCurrency = price_page['data']['offers']['priceCurrency']
    return price, priceCurrency

In [6]:
def get_shops_page(siteId, cityId, productId):
    headers= {
        'Origin': 'https://www.dns-shop.ru',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
    }

    body= {
        "siteId":siteId,
        "cityId":cityId,
        "products":[{"id":productId,"count":1}],
        "offset":0,
        "notShowAvails":False,
        "isCheckout":False,
        "locales":"ru",
        "isParentCityNeeded":False,
    }

    response = requests.post(shops_url, headers=headers, json=body)
    
    if response.status_code != 200:
        raise Exception(f'smth wrong with shops request: Status Code is {response.status_code}') 

    return response.json()
    

In [7]:
def get_shops(shops_page, region):
    shops = dict()
    quantity_total = 0
    for shop in shops_page['avail']['products'][0]['branches']:
        if shop['maxCount'] > 0:
            quantity_total += shop['maxCount']
            shops[shop['branchId']] = {'quantity': shop['maxCount']}
    for shop in shops_page['shops']:
        shop_id = shop['id']
        if shop_id not in shops.keys():
            continue
        new_keys = ['id', 'title', 'address', 'latitude', 'longitude', 'cityName']
        for new_key in new_keys:
            shops[shop_id][new_key] = shop[new_key]
        shops[shop_id]['is_suburb'] = (shop['cityName'] != region)

    return list(shops.values()), quantity_total

In [8]:
def get_product_info(driver, url):
    product_info = {}
    product_info['url'] = url
    
    try:
        del driver.requests
        driver.get(url)

        microdata_request = None

        waiting_time = 50
        sleep_time = 1
        while(microdata_request == None):
            if waiting_time <= 0:
                raise Exception('waiting too long for microdata request')
            waiting_time -= sleep_time
            sleep(sleep_time)
            for i in driver.requests:
                if ('/microdata/' in i.url) and not (i.response is None):
                    microdata_request = i
                    
        productId = microdata_request.url.split('/')[-2]
                    
        microdata_response = \
            json.loads(decode(microdata_request.response.body, 
                              microdata_request.response.headers.get('Content-Encoding', 'identity')).decode('utf-8'))


        product_info['artikul'] = microdata_response['data']['sku']
        

        product_info['info_by_region'] = {}

        for region in regions:
            price_page = get_price_page(microdata_request, city_paths[region])
            price, priceCurrency = get_price(price_page)
            
            shops_page = get_shops_page(siteId, cityIds[region], productId)
            shops, quantity_total = get_shops(shops_page, region)
            
            product_info['info_by_region'][region] = {
                'price' : price, 'priceCurrency' : priceCurrency, 
                'quantity' : shops, 'quantity_total' : quantity_total
            }
    except Exception as e:
        product_info['errors'] = e
    finally:
        return product_info

In [9]:
def get_products_info(urls):
    attempts_count = 1
    total_start_time = time.time()
    products_info = []
    driver = create_driver()
    for url in tqdm(urls):
        product_info = None
        for attempt in range(attempts_count):
            product_info = get_product_info(driver, url)
            if 'errors' not in product_info.keys():
                break
        products_info.append(product_info)
    driver.quit()
    print("--- Total: %s seconds ---" % (time.time() - total_start_time))
    return products_info

In [10]:
urls = ['https://www.dns-shop.ru/product/0025277114ce3333/plita-kompaktnaa-elektriceskaa-kitfort-kt-125-cernyj/', 'https://www.dns-shop.ru/product/006e8a0b3c0a3332/robot-pylesos-dexp-lf200-belyj/', 'https://www.dns-shop.ru/product/011025865db73330/mikrofon-greenbean-voicehead-vh2-flesh-s-jack-rozovyj/', 'https://www.dns-shop.ru/product/02042da9ac893330/hlebopec-endever-skyline-mb-61-belyj/', 'https://www.dns-shop.ru/product/04e64465557d3332/mikrofon-zet-gaming-koradji-quatro-belyj/', 'https://www.dns-shop.ru/product/05efeac03db43333/otparivatel-vertikalnyj-kitfort-kt-979-krasnyj/', 'https://www.dns-shop.ru/product/06b5dd96dcabed20/mikrofon-synco-lav-s8-cernyj/', 'https://www.dns-shop.ru/product/08b8ed97ac15ed20/mikrofon-behringer-ta-312s-cernyj/', 'https://www.dns-shop.ru/product/08bd693ded633361/hlebopec-redmond-rbm-1908-belyj/', ]
sample_size = 2
urls_test = urls

In [11]:
products_info = get_products_info(urls_test)

  0%|          | 0/9 [00:00<?, ?it/s]

--- Total: 30.41555619239807 seconds ---


In [12]:
# проверка, есть ли ошибки
for i in range(sample_size):
    if 'errors' in products_info[i].keys():
        print(i)

In [13]:
# преобразование в json
ans = {}
for product in products_info:
    if 'url' in product.keys():
        ans[product['url']] = product
        
with open('ans.txt', 'w', encoding='utf-8') as outfile:
    json.dump(ans, outfile, ensure_ascii=False)

## Пример параллельного запуска

In [14]:
def run_parallel(n_workers, urls):
    urls_blocks = np.array_split(urls, n_workers)
    out = []

    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        parsing_futures = (executor.submit(get_products_info, urls_block) for urls_block in urls_blocks)
        for future in concurrent.futures.as_completed(parsing_futures):

            out += future.result() 

    return out

In [16]:
sample_size = 100
n_workers = 3
urls_test = urls
out = run_parallel(n_workers, urls_test)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

--- Total: 33.42976999282837 seconds ---
--- Total: 36.562315940856934 seconds ---
--- Total: 36.661962032318115 seconds ---


In [ ]:
for i in range(10):
    if 'errors' in out[i].keys():
        print(i)